In [1]:
from datetime import datetime, timedelta
import json
import pandas as pd
data = pd.read_excel('./기상청41_단기예보 조회서비스_오픈API활용가이드_241128/기상청41_단기예보 조회서비스_오픈API활용가이드_격자_위경도(2411).xlsx')
data.head()


,구분,행정구역코드,1단계,2단계,3단계,격자 X,격자 Y,경도(시),경도(분),경도(초),위도(시),위도(분),위도(초),경도(초/100),위도(초/100),위치업데이트
0,kor,1100000000,서울특별시,NaN,NaN,60,127,126,58,48.03,37,33,48.85,126.980008,37.563569,NaN
1,kor,1111000000,서울특별시,종로구,NaN,60,127,126,58,53.91,37,34,13.36,126.981642,37.570378,NaN
2,kor,1111051500,서울특별시,종로구,청운효자동,60,127,126,58,14.35,37,35,2.89,126.970652,37.584137,NaN
3,kor,1111053000,서울특별시,종로구,사직동,60,127,126,58,15.44,37,34,23.77,126.970956,37.573269,NaN
4,kor,1111054000,서울특별시,종로구,삼청동,60,127,126,59,2.32,37,34,56.73,126.983978,37.582425,NaN


In [2]:
serviceKey = "SLnUCvSqwv%2FeT7pxQ2NFc2etMr8%2B7HLq1zXs6Djr1%2BXDUA%2F4YQvKK7C3nhGjIjOX0Vs42IoiIg%2FEQgHpcP371w%3D%3D" # 본인의 서비스 키 입력
# --> 날씨를 알고 싶은 시간 입력
date = (datetime.today() - timedelta(hours=1))
base_date = date.strftime("%Y%m%d") # 발표 일자
base_time = date.strftime("%H00") # 발표 시간
nx = '60' # 예보 지점 x좌표
ny = '127' # 예보 지점 y좌표

In [3]:
# 알고 싶은 시간
input_d = datetime.strptime(base_date + base_time, "%Y%m%d%H%M" )
print(input_d)

# 실제 입력 시간
input_d = datetime.strptime(base_date + base_time, "%Y%m%d%H%M" ) + timedelta(hours=1)
print(input_d)

input_datetime = datetime.strftime(input_d, "%Y%m%d%H%M")
input_date = input_datetime[:-4]
input_time = input_datetime[-4:]

2025-06-02 10:00:00
2025-06-02 11:00:00


In [4]:
# url
#url = "https://apis.data.go.kr/1360000/VilageFcstInfoService_2.0"
#params = {"serviceKey":"SLnUCvSqwv/eT7pxQ2NFc2etMr8+7HLq1zXs6Djr1+XDUA/4YQvKK7C3nhGjIjOX0Vs42IoiIg/EQgHpcP371w==","numOfRows":"1000","pageNo":"1","dataType":"JSON","base_date":input_date,"base_time":input_time,"nx":nx,"ny":ny}
url = f"http://apis.data.go.kr/1360000/VilageFcstInfoService_2.0/getUltraSrtNcst?serviceKey={serviceKey}&numOfRows=12&pageNo=1&dataType=json&base_date={base_date}&base_time={base_time}&nx={nx}&ny={ny}"
print(url)

http://apis.data.go.kr/1360000/VilageFcstInfoService_2.0/getUltraSrtNcst?serviceKey=SLnUCvSqwv%2FeT7pxQ2NFc2etMr8%2B7HLq1zXs6Djr1%2BXDUA%2F4YQvKK7C3nhGjIjOX0Vs42IoiIg%2FEQgHpcP371w%3D%3D&numOfRows=12&pageNo=1&dataType=json&base_date=20250602&base_time=1000&nx=60&ny=127


In [5]:
# url로 API return값 요청
import requests

requests.get(url)
response = requests.get(url)
res = json.loads(response.text)

In [6]:
res

{'response': {'header': {'resultCode': '00', 'resultMsg': 'NORMAL_SERVICE'},
  'body': {'dataType': 'JSON',
   'items': {'item': [{'baseDate': '20250602',
      'baseTime': '1000',
      'category': 'PTY',
      'nx': 60,
      'ny': 127,
      'obsrValue': '0'},
     {'baseDate': '20250602',
      'baseTime': '1000',
      'category': 'REH',
      'nx': 60,
      'ny': 127,
      'obsrValue': '63'},
     {'baseDate': '20250602',
      'baseTime': '1000',
      'category': 'RN1',
      'nx': 60,
      'ny': 127,
      'obsrValue': '0'},
     {'baseDate': '20250602',
      'baseTime': '1000',
      'category': 'T1H',
      'nx': 60,
      'ny': 127,
      'obsrValue': '21.1'},
     {'baseDate': '20250602',
      'baseTime': '1000',
      'category': 'UUU',
      'nx': 60,
      'ny': 127,
      'obsrValue': '1.9'},
     {'baseDate': '20250602',
      'baseTime': '1000',
      'category': 'VEC',
      'nx': 60,
      'ny': 127,
      'obsrValue': '279'},
     {'baseDate': '20250602',
   

In [6]:
res['response']['body']['items']['item']

KeyError: 'body'

In [33]:
informations = dict()
for items in res['response']['body']['items']['item'] :
    cate = items['category']
    fcstTime = items['fcstTime']
    fcstValue = items['fcstValue']
    temp = dict()
    temp[cate] = fcstValue
    
    if fcstTime not in informations.keys() :
        informations[fcstTime] = dict()
#     print(items['category'], items['fcstTime'], items['fcstValue'])
#     print(informations[fcstTime])
    informations[fcstTime][cate] = fcstValue

print(informations)

{'0000': {'TMP': '17', 'UUU': '-1.1', 'VVV': '1.5', 'VEC': '141', 'WSD': '1.9', 'SKY': '1', 'PTY': '0', 'POP': '0', 'WAV': '-999', 'PCP': '강수없음', 'REH': '55', 'SNO': '적설없음'}, '0100': {'TMP': '16', 'UUU': '-1.8', 'VVV': '2.4', 'VEC': '142', 'WSD': '3', 'SKY': '1', 'PTY': '0', 'POP': '0', 'WAV': '-999', 'PCP': '강수없음', 'REH': '55', 'SNO': '적설없음'}, '0200': {'TMP': '16', 'UUU': '-1.6', 'VVV': '2.3', 'VEC': '145', 'WSD': '2.9', 'SKY': '1', 'PTY': '0', 'POP': '0', 'WAV': '-999', 'PCP': '강수없음', 'REH': '60', 'SNO': '적설없음'}, '0300': {'TMP': '15', 'UUU': '-1.7', 'VVV': '2', 'VEC': '139', 'WSD': '2.7', 'SKY': '1', 'PTY': '0', 'POP': '0', 'WAV': '-999', 'PCP': '강수없음', 'REH': '55', 'SNO': '적설없음'}, '0400': {'TMP': '15', 'UUU': '-2', 'VVV': '2.1', 'VEC': '135', 'WSD': '3', 'SKY': '1', 'PTY': '0', 'POP': '0', 'WAV': '-999', 'PCP': '강수없음', 'REH': '60', 'SNO': '적설없음'}, '0500': {'TMP': '14', 'UUU': '-2.2', 'VVV': '2.4', 'VEC': '136', 'WSD': '3.3', 'SKY': '1', 'PTY': '0', 'POP': '0', 'WAV': '-999', 'PCP': 

In [34]:
deg_code = {0 : 'N', 360 : 'N', 180 : 'S', 270 : 'W', 90 : 'E', 22.5 :'NNE',
           45 : 'NE', 67.5 : 'ENE', 112.5 : 'ESE', 135 : 'SE', 157.5 : 'SSE',
           202.5 : 'SSW', 225 : 'SW', 247.5 : 'WSW', 292.5 : 'WNW', 315 : 'NW',
           337.5 : 'NNW'}

def deg_to_dir(deg) :
    close_dir = ''
    min_abs = 360
    if deg not in deg_code.keys() :
        for key in deg_code.keys() :
            if abs(key - deg) < min_abs :
                min_abs = abs(key - deg)
                close_dir = deg_code[key]
    else : 
        close_dir = deg_code[deg]
    return close_dir
deg_to_dir(0)

'N'

In [35]:
pyt_code = {0 : '강수 없음', 1 : '비', 2 : '비/눈', 3 : '눈', 5 : '빗방울', 6 : '진눈깨비', 7 : '눈날림'}
sky_code = {1 : '맑음', 3 : '구름많음', 4 : '흐림'}


for key, val in zip(informations.keys(), informations.values()) :
#     print(key, val)
    # val['LGT'] -- 낙뢰 
    template = f"""{base_date[:4]}년 {base_date[4:6]}월 {base_date[-2:]}일 {key[:2]}시 {key[2:]}분 {(int(nx), int(ny))} 지역의 날씨는 """ 
    
    
    # 맑음(1), 구름많음(3), 흐림(4)
    if val['SKY'] :
        sky_temp = sky_code[int(val['SKY'])]
#         print("하늘 :", sky_temp)
        template += sky_temp + " "
    
    # (초단기) 없음(0), 비(1), 비/눈(2), 눈(3), 빗방울(5), 빗방울눈날림(6), 눈날림(7)
    if val['PTY'] :
        pty_temp = pyt_code[int(val['PTY'])]
#         print("강수 여부 :",pty_temp)
        template += pty_temp
        # 강수 있는 경우
        if val['RN1'] != '강수없음' :
            # RN1 1시간 강수량 
            rn1_temp = val['RN1']
#             print("강수량(1시간당) :",rn1_temp)
            template += f"시간당 {rn1_temp}mm "
    
    # 기온
    if val['T1H'] :
        t1h_temp = float(val['T1H'])
#         print(f"기온 : {t1h_temp}℃")
        template += f" 기온 {t1h_temp}℃ "
    # 습도
    if val['REH'] :
        reh_temp = float(val['REH'])
#         print(f"습도 : {reh_temp}%")
        template += f"습도 {reh_temp}% "
    # val['UUU'] -- 바람
    
    # val['VVV'] -- 바람
    
    # 풍향/ 풍속
    if val['VEC'] and val['WSD']:
        vec_temp = deg_to_dir(float(val['VEC']))
        wsd_temp = val['WSD']
#         print(f"풍속 :{vec_temp} 방향 {wsd_temp}m/s")
        
    template += f"풍속 {vec_temp} 방향 {wsd_temp}m/s"
    print(template)

KeyError: 'RN1'

In [7]:
url = f"http://apis.data.go.kr/1360000/VilageFcstInfoService_2.0/getFcstVersion?serviceKey={serviceKey}&numOfRows=12&pageNo=1&dataType=json&base_time={base_time}&ftype=SHRT"

In [8]:
response = requests.get(url)

In [9]:
response

<Response [200]>